# Constraints
Generally, a vector-valued constraints function, $g$, defines a solution $x$ as feasible if and only if $g_i(x) \le 0$ for all $i$. In order to optimize a function with some constraints, we use the class `ConstrainedFitnessAL` which constructs an Augmented Lagrangian objective function from the objective and arbitrary non-linear constraints.

For some positive and changing parameters $\lambda_i, \mu_i$, the constructed function is

$$x\mapsto f(x) + \sum_i (\lambda_i g_i + \frac{\mu_i}{2}g_i^2)$$

where $g_i := \max\left(g_i(x), -\frac{\lambda_i}{\mu_i}\right)$. This is not expected to work well if the constraint is a step function at the feasibility boundary.

For simplicity, let's consider a constraint that restricts the second variable to values $\le-1$. 


In [ ]:
import cma

fun = cma.ff.elli  # we could use `functools.partial(cma.ff.elli, cond=1e4)` to change the condition number to 1e4
def constraints(x):
    return [x[1] - -1, x[1] - -0.9]  # constrain the second variable to <= -1, the second constraint is superfluous
cfun = cma.ConstrainedFitnessAL(fun, constraints)  # unconstrained function with adaptive Lagrange multipliers

x0 = 4 * [2]  # initial solution
sigma0 = 1    # initial standard deviation to sample new solutions

x, es = cma.fmin2(cfun, x0, sigma0, {'tolstagnation': 0}, callback=cfun.update)
x = es.result.xfavorite  # the original x-value may be meaningless
constraints(x)  # show constraint violation values

The solution `es.result.xfavorite` is expected to be close to the feasible domain, but it may not be "stricly" feasible. To find a truly feasible solution, we can run a post-optimization like

In [ ]:
c = es.countiter
x = cfun.find_feasible(es)
print("find_feasible took {} iterations".format(es.countiter - c))
constraints(x)  # is now <= 0

We can also plot the time evolution of feasibility and Augmented Lagrangian parameters and, of course, the standard plot: 

In [ ]:
# "%matplotlib widget" gives interactive inline plots (pip install --upgrade jupyterlab ipympl); CAVEAT: it does not open a new figure automatically
# fix failing variable completion: %config Completer.use_jedi = False

In [ ]:
es.plot(xsemilog=True, x_opt=[0, -1] + (len(x0) - 2) * [0]);

In [ ]:
cfun.al.loggers.plot()

The `ConstrainedFitnessAL` class instance provides some more detailed information:

In [ ]:
cfun.archives[0].archive  # non-dominated list of [f, aggregate(g)], feasible if aggregate(g) <= 0

In [ ]:
cfun.archives[0].archive.infos[-1]

Details about the best feasible solution can be found under its `info` attribute.


In [ ]:
cfun.best_feas.info

# Ask-and-Tell Interface
We can use the underlying `CMAEvolutionStrategy` class, on which `cma.fmin2` is based, to get more control over the optimization loop.

In [ ]:
# a more verbose way to run ``_, es = cma.fmin2(cfun, x0, sigma0)``

cfun = cma.ConstrainedFitnessAL(fun, constraints)  # unconstrained function with adaptive Lagrange multipliers
es = cma.CMAEvolutionStrategy(x0, sigma0)

while not es.stop():
    X = es.ask()  # sample len(X) candidate solutions
    es.tell(X, [cfun(x) for x in X])
    cfun.update(es)
    es.logger.add()  # for later plotting
    es.disp()
es.result_pretty()

If the function or constraints take additional arguments, they can be assigned ahead of time using [`functools.partial`](https://docs.python.org/3/library/functools.html), even to a mutable data type which makes it possible to change them dynamically.

In [ ]:
cma.plot();